#### Bridget Sands
#### Harvard University
#### Applied Mathematics Senior Thesis
#### April 1st, 2024

# "Data_Cleaning_SB.ipynb"

### Note: This is the 2nd(B) notebook used for cleaning, following the first cleaning notebook that adjusts for the men on base. It is denoted (B) because it properly prepares data for the SB model. (A) prepares data for the PA model.

#### Notebook Purpose and Summary:
This notebook was used to do the final cleaning and preparation of the data that had already been cleaned from the first data cleaning notebook, `Clean_OB.ipynb`. This notebook takes in a season of cleaned data, does additional featuring engineering, and exports the season ready to be imported into the SB model.

#### Input:
1. `csv` season of data for specific league/year, already cleaned by the `Clean_OB.ipynb`.
2. `problem_pks.csv` of game_pks for games where ABS is used.

#### Export:
1. `csv` season of data for inputted specific league/year, ready imported into the `SB_model.Rmd` file.
2. `csv` of unique batter ids and names from this season of data.
3. `csv` of unique pitcher ids and names from this season of data.
4. `csv` of unique ids and names for man on first base from this season of data.

#### Glossary:
- PA: Plate appearance

#### Additional Notes:
- Following data cleaning and investigation, it is clear that columns that begin with `details.` generally describe the specific row entry, while columns that begin with `results.` provide information about the overall PA the entry belongs to.

In [1]:
# Import helpful libraries
import numpy as np
import pandas as pd
import math

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Remember to CHANGE FILE:
#### Relative to season needed to clean

In [4]:
# Read in file as a pandas dataframe
df = pd.read_csv('da14_wOB_F.csv', low_memory=False)

In [4]:
# Sort values by game, inning, half inning, atBatIndex, and startTime
df = df.sort_values(by=['game_pk', 'about.inning', 'about.halfInning', 'about.atBatIndex', 'startTime'], ignore_index=True)

In [5]:
# Check how many SBAs and pickoff attempts have NA startTimes 
# Should be as low as possible, or else big issues, out of order
steals = ['Stolen Base 2B', 'Caught Stealing 2B', 'Pickoff Caught Stealing 2B']
df[(df['details.event'].isin(steals))|(df['details.description']=='Pickoff Attempt 1B')]['startTime'].isna().sum()

0

In [6]:
# Consider different situations in which SBAs or pickoffs occur
# They should all have men on base in some capacity --> can't steal if no one is on base
df[((df['type']!='pitch')&((df['details.event'].isin(steals))|((df['details.description']=='Pickoff Attempt 1B')&(df['result.eventType'].str.contains('caught_stealing_2b')))))]['Men_OB'].value_counts()

Men_OB
Men_On    2567
RISP       483
Name: count, dtype: int64

In [7]:
# Isolate the PA ids in which there are men on base
pas = df[df['Men_OB'].isin(['Men_On', 'RISP', 'Loaded'])]['PA_id'].unique()
print(len(pas))

# Set temporary values for firstID and firstName to represent 
# batter id and name of first baserunner
df['firstID'] = np.where(df['PA_id'].isin(pas), 'AGH', '')
df['firstName'] = np.where(df['PA_id'].isin(pas), 'AGH', '')

79585


In [8]:
# Inspect the PA to confirm proper format
pas

array(['716352-1-1-1', '716352-2-1-1', '716352-3-1-1', ...,
       '718782-80-9-1', '718782-82-9-1', '718782-83-9-1'], dtype=object)

In [9]:
# Print the length of the unique PA list
len(pas)

79585

## Find identification for the man on first:

In [12]:
# Error count that increments if a PA is considered without a man on base
errors = 0

# Error that increments and prints if there is a case that is not bucketed 
errors2 =0

# Count the iteration
cnt = 0 

# Call to iteration 
for pa in pas:
    cnt += 1

    # If divisible by 1000, print count--> progress tracking
    if cnt % 1000 == 1:
        print(cnt)

    # Gain access to PA before current
    t = df[df['PA_id'] == pa].index.min()-1

    # Pull necessary information from PA before
    a, b, c, d = df.loc[t, ['Men_OB', 'matchup.splits.menOnBase', 'result.eventType', 'result.description']].values

    # Confirm last PA resulted in at least one man on base
    if (b != 'Men_On') and (b != 'RISP') and (b != 'Loaded'):
        errors += 1
        continue

    # Handle case in which batter is always the one to get on base
    if c in ['single', 'walk', 'intent_walk', 'field_error', 'hit_by_pitch', 'catcher_interf', 'fielders_choice', 'fielders_choice_out']:
        df.loc[df['PA_id']==pa,['firstID', 'firstName']] = df.loc[t,['matchup.batter.id', 'matchup.batter.fullName']].values

    # Handle force out case
    elif c =='force_out':
        if ("to 1st" not in d) and ("to 2nd" in d):
            df.loc[df['PA_id']==pa,['firstID', 'firstName']] = ['', '']
        else:
            df.loc[df['PA_id']==pa,['firstID', 'firstName']] = df.loc[t,['matchup.batter.id', 'matchup.batter.fullName']].values

    # Handle double play grounders
    elif c == 'grounded_into_double_play':
        if ('out at 1st' in d):
            df.loc[df['PA_id']==pa,['firstID', 'firstName']] = ['', '']
        else:
            df.loc[df['PA_id']==pa,['firstID', 'firstName']] = df.loc[t,['matchup.batter.id', 'matchup.batter.fullName']].values

    # Handle double plays
    elif c == 'double_play':
        if 'to 1st' in d:
            df.loc[df['PA_id']==pa,['firstID', 'firstName']] = df.loc[t,['matchup.batter.id', 'matchup.batter.fullName']].values
        elif ("to 2nd" in d) or ("out at 1st" in d):
            df.loc[df['PA_id']==pa,['firstID', 'firstName']] = ['', '']
        else:
            df.loc[df['PA_id']==pa,['firstID', 'firstName']] = df.loc[t, ['firstID', 'firstName']].values

    # Handle sacrifice bunts
    elif c == 'sac_bunt':
        if 'to 1st' in d:
            df.loc[df['PA_id']==pa,['firstID', 'firstName']] = df.loc[t,['matchup.batter.id', 'matchup.batter.fullName']].values
        else:
            df.loc[df['PA_id']==pa,['firstID', 'firstName']] = ['', '']

    # Handle field outs
    elif c == 'field_out':
        if (any(substring in d for substring in ['flies', 'pops', 'lines', 'grounds out'])) and ('to 2nd' not in d):
            df.loc[df['PA_id']==pa,['firstID', 'firstName']] = df.loc[t, ['firstID', 'firstName']].values
        
        else:
            df.loc[df['PA_id']==pa,['firstID', 'firstName']] = ['', '']

    # Handle sacrifice outs
    elif c in ['sac_fly', 'sac_fly_double_play']:
        if 'to 1st' in d:
            df.loc[df['PA_id']==pa,['firstID', 'firstName']] = df.loc[t,['matchup.batter.id', 'matchup.batter.fullName']].values

        elif 'to 2nd' in d:
            df.loc[df['PA_id']==pa,['firstID', 'firstName']] = ['', '']

        else: 
            df.loc[df['PA_id']==pa,['firstID', 'firstName']] = df.loc[t, ['firstID', 'firstName']].values
            
    # Handle strikeouts
    elif c == 'strikeout':
        if "to 1st" in d:
            df.loc[df['PA_id']==pa,['firstID', 'firstName']] = df.loc[t,['matchup.batter.id', 'matchup.batter.fullName']].values
        else: 
            df.loc[df['PA_id']==pa,['firstID', 'firstName']] = df.loc[t, ['firstID', 'firstName']].values

    # Handle cases in which batter is never on first
    elif c in ['double', 'triple']:
        df.loc[df['PA_id']==pa,['firstID', 'firstName']] = ['', '']

    # Print out error if it doesn't fall into any of the considered caxes
    else:
        print('Error! Beware.')
        errors2 += 1

1
1001
2001
3001
4001
5001
6001
7001
8001
9001
10001
11001
12001
13001
14001
15001
16001
17001
18001
19001
20001
21001
22001
23001
24001
25001
26001
27001
28001
29001
30001
31001
32001
33001
34001
35001
36001
37001
38001
39001
40001
41001
42001
43001
44001
45001
46001
47001
48001
49001
50001
51001
52001
53001
54001
55001
56001
57001
58001
59001
60001
61001
62001
63001
64001
65001
66001
67001
68001
69001
70001


KeyboardInterrupt: 

In [ ]:
# Check how many entries don't have a row for the man on first
# Note that if there isn't anyone on first, there should be an empty string
# Therefore this value should be as close to zero as possible
len(df[df['firstID']=='AGH'])

In [ ]:
# Check number of entries known to have a man on first but don't have entries
# Number wanted to be as close to zero as possible
len(df[(df['firstID']=='')&(df['Men_OB']=='Men_On')]) 

In [ ]:
# Display all incorrect cases
df[(df['firstID']=='')&(df['Men_OB']=='Men_On')]

In [ ]:
# Create last pitch indicator to be used in further feature engineering
max_vals = df.groupby('PA_id')['pitchNumber'].transform('max') == df['pitchNumber']
df['Last_pitch'] = max_vals.astype(int)

# First part of Strikes feature engineering
df['Strikes'] = np.where((df['type']=='pitch')&(df['details.isStrike'])&(df['count.strikes.start']==2)&(df['Last_pitch']==False)&(df['details.description'].isin(['Foul', 'Foul Bunt', 'Foul Tip'])), -1,
                                                                                                            np.where((df['type']=='pitch')&df['details.isStrike'], df['count.strikes.start']-1, df['count.strikes.start']))
                                                                                                             

# Balls feature engineering
df['Balls'] = np.where(df['details.isBall']==True, df['count.balls.start']-1, df['count.balls.start'])

# Rename Outs feature
df['Outs'] = df['count.outs.start']

# Create Batter_home feature
df['Batter_home'] = np.where(df['about.isTopInning'], 0, 1)

# Rename Inning feature
df['Inning'] = df['about.inning']

In [ ]:
# Understand strikes after first part of engineering
df['Strikes'].value_counts()

In [ ]:
# Run second parts of Strikes feature engineering
for index, row in df[df['Strikes']==-1].iterrows():
    r = row['count.strikes.start']
    curr_loc = df.index.get_loc(index)
    nxt = df.loc[(index-10):index-1][df.loc[(index-10):index-1, 'type'] == 'pitch'].index.max()
    #print(index)
    b4 = df.loc[nxt, 'count.strikes.start']
    if r == b4:
        df.loc[index, 'Strikes2'] = row['count.strikes.start']
    else:
        #print('penis')
        df.loc[index, 'Strikes2'] = row['count.strikes.start']-1

In [ ]:
# Understand second part of Strikes engineering outcome
print(df['Strikes2'].value_counts())

# Execute final part of Strikes engineering
df['Strikes'] = np.where(df['Strikes']==-1, df['Strikes2'], df['Strikes'])
df['Strikes'].value_counts()

In [ ]:
# Investigate NAs in the data
df.isna().sum()

In [ ]:
# Isolate PAs in which SBA or pickoff variation occurred
unis = df[(df['Men_OB'] == 'Men_On')&((df['details.event'].isin(steals))|(df['details.description']=='Pickoff Attempt 1B'))]['PA_id'].unique()
# Print length of this 
len(unis)

## Create response variable and AB_pickoffs count:

In [ ]:
# Initialize values for features to be created
df['AB_pickoffs'] = 0
df['SBA'] = 0
df['SBA_spec'] = ''
df['SBA_check'] = 0

# Intitialize counter to keep track of iterations
c = 0

# Initialize counter that keeps track of number of PAs skipped (in original list by error)
skip = 0

# Iteratie through the list
for id2 in unis:
    c += 1

    # Print progress if iteration is divisiable --> tracking
    if c % 1000 == 1:
        print(c)

    # Create temporary dataframe for the PA
    temp = df[df['PA_id']==id2].copy()

    # Check if current PA was added in error
    if ('Pickoff Attempt 1B' not in temp['details.description'].values) and (len(temp[temp['details.event'].isin(steals)])==0):
        skip += 1
        continue

    # Initialize AB_pickoffs for current PA 
    po_c = 0

    # Iterate through each row of temporary dataframe 
    for index, row in temp.iterrows():

        # If the current row is a pickoff attempt, add to AB_pickoffs count
        if (row['details.description'] == 'Pickoff Attempt 1B') and ('caught_stealing_2b' not in row['result.eventType']):
            po_c += 1
        
        # Set AB_pickoffs for each row 
        df.loc[index, 'AB_pickoffs'] = po_c

        # Identify instances where half inning ends on a caught SBA
        if ((row['details.description'] == 'Pickoff Attempt 1B') and ('caught_stealing_2b' in row['result.eventType'])):
            max_index = temp['pitchNumber'].idxmax()

            # Try to associate with the pitch before, if possible
            if not math.isnan(max_index):
                df.loc[max_index, 'SBA_spec'] = row['result.eventType']
                df.loc[max_index, 'SBA'] = 1
                df.loc[index, 'SBA_check'] = 1

            # Create fake pitch to associate otherwise
            else:
                if index == temp.index.max():
                    df.loc[index, 'type'] = 'pitch'
                    df.loc[index, 'AB_pickoffs'] = index - temp.index.min()
                    df.loc[index, 'SBA_spec'] = row['result.eventType']
                    df.loc[index, 'SBA'] = 1
                    df.loc[index, 'SBA_check'] = 1

        # Consider cases where SBA that doesn't end half inning
        elif (row['details.event'] in steals):
            
            # Associate with pitch before if possible
            next_pitch_index = temp.loc[(index-1):index][temp.loc[(index-1):index, 'type'] == 'pitch'].index.max()
            if not pd.isna(next_pitch_index):
                if df.loc[next_pitch_index, 'PA_id'] == row['PA_id']:
                    if (row['details.event'] in ['Caught Stealing 2B', 'Pickoff Caught Stealing 2B']):
                        df.loc[next_pitch_index, 'Outs'] = row['Outs']-1 
                    #print('YA')
                    df.loc[next_pitch_index, 'SBA_spec'] = row['details.event']
                    df.loc[next_pitch_index, 'SBA'] = 1
                    df.loc[index, 'SBA_check'] = 1

            # If not associate with pitch after 
            else: 
                last_pitch_index = df.loc[(index+1):][df.loc[(index+1):, 'type'] == 'pitch'].index.min()
                if not pd.isna(last_pitch_index):
                    if (row['details.event'] in ['Caught Stealing 2B', 'Pickoff Caught Stealing 2B']):
                        df.loc[last_pitch_index, 'Outs'] = row['Outs']-1 
                    if (row['about.inning'] == df.loc[last_pitch_index, 'about.inning']) and (row['about.halfInning'] == df.loc[last_pitch_index, 'about.halfInning']):
                        df.loc[last_pitch_index, 'SBA_spec'] = row['details.event']
                        df.loc[last_pitch_index, 'SBA'] = 1
                        df.loc[index, 'SBA_check'] = 1


In [ ]:
# Confirm no rows with the Balls feature = -1
df['Balls'].value_counts()

In [ ]:
# Confirm no rows with the Outs feature = -1
df['Outs'].value_counts()

In [22]:
# Evaluate the distribution of AB_pickoffs for pitches that do not have SBA
df[df['SBA']==0]['AB_pickoffs'].value_counts()

KeyError: 'SBA'

In [ ]:
# Print number of skipped PAs (previously chosen in error)
print(skip)

In [21]:
# Consider the relative success of correctly identified SBAs
# Note this also includes all of the pickoffs which are not real SBAs
print(sum(df['SBA']))
print(len(df[(df['Men_OB']=='Men_On')&(df['type']!='pitch')&((df['details.event'].isin(steals))|((df['details.description']=='Pickoff Attempt 1B')&(df['result.eventType'].str.contains('caught_stealing_2b')))]))

2640
2657


In [23]:
# Print the length of the entire df
print(len(df))
# Filter to only consider situations with a singular runner on first
df = df[df['Men_OB']=='Men_On']
# Print length of df post filter
print(len(df))

807669
166744


In [24]:
# Further filter so df only includes pitches, the SBAs now associated with pitches
df = df[df['type']=='pitch']
# Print new length of df
len(df)

143744

In [25]:
# Consider response variable count
df['SBA'].value_counts()

2638

## Remember to adjust GRACE PERIOD:
#### Note:
If data is from Double-A or Triple-A in 2022, they started the season with a "Grace Period" relative to the Pitch Timer. Filter to only include games after this period.

In [28]:
# # 2022 GRACE PERIOD AAA AND AA
# df['game_date'] = pd.to_datetime(df['game_date'])
# df = df[df['game_date']>='2022-04-15']
# sorted(df['game_date'].values)[0]

In [26]:
# Rename columns for export
rename = {'matchup.batter.id':'Batter_id', 'matchup.batter.fullName':'Batter_name', 
          'matchup.pitcher.id':'Pitcher_id', 'matchup.pitcher.fullName':'Pitcher_name'}

df = df.rename(columns=rename)

In [27]:
# Create final df
f_cols = ['game_pk','Strikes', 'Balls', 'Outs', 'Batter_home', 'Inning',
          'firstID',  'Batter_id', 'Pitcher_id', 'AB_pickoffs', 'SBA_check', 'SBA_spec', 'SBA']
df_f = df[f_cols].copy()

## Remember to CHANGE YEAR AND LEAGUE:
#### Relative to season cleaning

In [29]:
# Set Year and League values
df_f['Year'] = 2014
df_f['League'] = 'AA'

## Remember to CHANGE TREATMENTS:
#### Relative to season cleaning

In [30]:
# Set rule implemenation status

# Pitch timer
# Control, v1, v2
df_f['Pitch_timer'] = 'Control'

# Bigger Bases
# 0, 1
df_f['Bigger_bases'] = 0

# Defensive shift limits 
# Control, v1, v2
df_f['Defensive_shift_limits'] = 'Control'

### ABS Adjustment:

In [31]:
# Read in problematic PAs
pks = pd.read_csv('problem_pks.csv', low_memory=False)

# Assign indicator for problematic
df_f['ABS'] = np.where((df_f['Year']==2023)&(df_f['League']=='AAA'), 1, np.where(df_f['game_pk'].isin(pks['value'].values), 1, 0))

# Evaluate values
df_f['ABS'].value_counts()

ABS
0    143744
Name: count, dtype: int64

In [32]:
# Create pitcher, batter, and first baserunner keys
pitcher_key = df[['Pitcher_name', 'Pitcher_id']].copy().drop_duplicates(ignore_index=True)
batter_key = df[['Batter_name', 'Batter_id']].copy().drop_duplicates(ignore_index=True)
first_key = df[['firstName', 'firstID']].copy().drop_duplicates(ignore_index=True)

In [33]:
# Print final length of df
len(df_f)

143744

## Write and export code to csv:
### Remember to CHANGE FILE LABELS.

In [34]:
df_f.to_csv('da14_SB.csv')
pitcher_key.to_csv('pitcher_key_SB_da14.csv')
batter_key.to_csv('batter_key_SB_da14.csv')
first_key.to_csv('first_key_SB_da14.csv')